In [1]:
# !wget https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q5_1.bin

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python
!pip install -q fastapi==0.101.0 uvicorn==0.13.1 langchain==0.0.234 colabcode==0.3.0

In [3]:
from colabcode import ColabCode
from fastapi import FastAPI
from pydantic import BaseModel
from langchain.llms import LlamaCpp


class LLAMA_MODEL:
    def __init__(self) -> None:
        # Make sure the model path is correct for your system!
        self.llm = LlamaCpp(
            model_path="./llama-2-13b-chat.ggmlv3.q5_1.bin",
            input={"temperature": 0.75, "max_length": 2000, "top_p": 1},
            n_gpu_layers=60,
            verbose=False,
        )

    def generate(self, prompt: str) -> str:
        return self.llm(prompt)

In [4]:
llm_model = LLAMA_MODEL()
app = FastAPI()
cc = ColabCode(port=12000, code=False)

llama.cpp: loading model from ./llama-2-13b-chat.ggmlv3.q5_1.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_head_kv  = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 1.0e-06
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 9 (mostly Q5_1)
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =    0.11 MB
llama_model_load_internal: mem required  = 9677.07 MB (+  400.00 MB per state)
llama_new_context_with_model: kv self size  =  400.00 MB


In [5]:
class UserInput(BaseModel):
    prompt: str


@app.post("/")
async def read_root(user_input: UserInput):
    output = llm_model.generate(user_input.prompt)
    return output

cc.run_app(app=app)